# Full Example: Predictions for HRES T0

In this example, we will download HRES T0 data for 11 May 2022 from WeatherBench2 at 0.25 degrees resolution and run Aurora on this data. We will use the version of Aurora that was fine-tuned on IFS HRES T0 in this example. 

Running this notebook requires installing an additional python package in order to download from Google Cloud Storage:

`pip install gcsfs`

In [1]:
from pathlib import Path
import xarray as xr
import fsspec
import gcsfs

# Data will be downloaded here.
download_path = Path("~/downloads")
download_path = download_path.expanduser()
download_path.mkdir(parents=True, exist_ok=True)

# Load outline of Zarr archive. 
url = "gs://weatherbench2/datasets/hres_t0/2016-2022-6h-1440x721.zarr"
ds = xr.open_zarr(fsspec.get_mapper(url), chunks=None)

time = '2022-05-11'

# Download the surface-level variables.
if not (download_path / f"{time}-surface-level.nc").exists():
    surface_vars = ["10m_u_component_of_wind", "10m_v_component_of_wind", "2m_temperature", "mean_sea_level_pressure"]
    ds_surf = ds[surface_vars].sel(time=time).compute()
    ds_surf.to_netcdf(str(download_path / f"{time}-surface-level.nc"))
print("Surface-level variables downloaded!")

# Download the atmospheric variables.
if not (download_path / f"{time}-atmospheric.nc").exists():
    atmos_vars = ["temperature", "u_component_of_wind", "v_component_of_wind", "specific_humidity", "geopotential"]
    ds_atmos = ds[atmos_vars].sel(time=time).compute()
    ds_atmos.to_netcdf(str(download_path / f"{time}-atmospheric.nc"))
print("Atmos-level variables downloaded!")



Surface-level variables downloaded!
Atmos-level variables downloaded!


## Downloading Static Variables from ERA5 Data
The static variables are not available in WeatherBench2, so we need to download them from ERA5 as in `example.ipynb`. To do so, register an account with the [Climate Data Store](https://cds.climate.copernicus.eu/) and create `$HOME/.cdsapirc` with the following content:

```
url: https://cds.climate.copernicus.eu/api/v2
key: <UID>:<API key>
```

You can find your UID and API key on your account page.

In [2]:
from pathlib import Path

import cdsapi

# Data will be downloaded here.
download_path = Path("~/downloads")

c = cdsapi.Client()

download_path = download_path.expanduser()
download_path.mkdir(parents=True, exist_ok=True)

# Download the static variables.
if not (download_path / "static.nc").exists():
    c.retrieve(
        "reanalysis-era5-single-levels",
        {
            "product_type": "reanalysis",
            "variable": [
                "geopotential",
                "land_sea_mask",
                "soil_type",
            ],
            "year": "2023",
            "month": "01",
            "day": "01",
            "time": "00:00",
            "format": "netcdf",
        },
        str(download_path / "static.nc"),
    )
print("Static variables downloaded!")

Static variables downloaded!


## Preparing a Batch

We convert the downloaded data to an `aurora.Batch`, which is what the model requires.

In [4]:
import torch
import xarray as xr

from aurora import Batch, Metadata

static_vars_ds = xr.open_dataset(download_path / "static.nc", engine="netcdf4")
surf_vars_ds = xr.open_dataset(download_path / f"{time}-surface-level.nc", engine="netcdf4")
atmos_vars_ds = xr.open_dataset(download_path / f"{time}-atmospheric.nc", engine="netcdf4")

i = 1  # Select this time index in the downloaded data.

batch = Batch(
    surf_vars={
        # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
        # batch dimension of size one.
        "2t": torch.from_numpy(surf_vars_ds["2m_temperature"].values[[i - 1, i]][None]),
        "10u": torch.from_numpy(surf_vars_ds["10m_u_component_of_wind"].values[[i - 1, i]][None]),
        "10v": torch.from_numpy(surf_vars_ds["10m_v_component_of_wind"].values[[i - 1, i]][None]),
        "msl": torch.from_numpy(surf_vars_ds["mean_sea_level_pressure"].values[[i - 1, i]][None]),
    },
    static_vars={
        # The static variables are constant, so we just get them for the first time.
        "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
    },
    atmos_vars={
        "t": torch.from_numpy(atmos_vars_ds["temperature"].values[[i - 1, i]][None]),
        "u": torch.from_numpy(atmos_vars_ds["u_component_of_wind"].values[[i - 1, i]][None]),
        "v": torch.from_numpy(atmos_vars_ds["v_component_of_wind"].values[[i - 1, i]][None]),
        "q": torch.from_numpy(atmos_vars_ds["specific_humidity"].values[[i - 1, i]][None]),
        "z": torch.from_numpy(atmos_vars_ds["geopotential"].values[[i - 1, i]][None]),
    },
    metadata=Metadata(
        lat=torch.from_numpy(surf_vars_ds.latitude.values * -1),    # Metadata expects latitudes to be in the range [90, -90].
        lon=torch.from_numpy(surf_vars_ds.longitude.values),
        # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        # one value for every batch element.
        time=(surf_vars_ds.time.values.astype("datetime64[s]").tolist()[i],),
        atmos_levels=tuple(int(level) for level in atmos_vars_ds.level.values),
    ),
)

/home/t-analucic/mambaforge/envs/climai_global/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading and Running the Model

Finally, we are ready to load and run the model and visualise the predictions. We perform a roll-out for two steps, which produces predictions for hours 12:00 and 18:00.

In [6]:
from aurora import Aurora, rollout

model = Aurora(use_lora=True)  # The 0.25 fine-tuned version uses LoRA.
model.load_checkpoint("wbruinsma/aurora", "aurora-0.25-finetuned.ckpt")

model.eval()
model = model.to("cuda")

with torch.inference_mode():
    preds = [pred.to("cpu") for pred in rollout(model, batch, steps=1)]

model = model.to("cpu")

aurora-0.25-finetuned.ckpt: 100%|██████████| 5.04G/5.04G [02:04<00:00, 40.6MB/s]
/home/t-analucic/mambaforge/envs/climai_global/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


type: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 2, figsize=(12, 6.5))

for i in range(ax.shape[0]):
    pred = preds[i]
    
    ax[i, 0].imshow(pred.surf_vars["2t"][0, 0].numpy() - 273.15, vmin=-50, vmax=50)
    ax[i, 0].set_ylabel(str(pred.metadata.time[0]))
    if i == 0:
        ax[i, 0].set_title("Aurora Prediction")
    ax[i, 0].set_xticks([])
    ax[i, 0].set_yticks([])
    
    ax[i, 1].imshow(surf_vars_ds["t2m"][2 + i].values - 273.15, vmin=-50, vmax=50)
    if i == 0:
        ax[i, 1].set_title("ERA5")
    ax[i, 1].set_xticks([])
    ax[i, 1].set_yticks([])

plt.tight_layout()